In [114]:
import sys
import numpy as np
import cv2
import matplotlib.pyplot as plt
from astropy.io import fits
sys.path.insert(0, 'C:\\Users\\EPL User\\Documents\\GitHub\wcmelby-HWP_Characterizations\\operational_scripts')
# from New_DRRP_Functions import *
from New_DRRP_Functions import extract_intensities, q_ultimate_polarimetry

In [ ]:
# TODO: check for oversaturated pixels and remove them
# TODO: correct dark subtraction for reduced images. Darks at this location: ""D:\desktop_drrp_data\test_12_17_2024\darks""
# TODO: automate data reduction process
# TODO: remove cutoff function?
# TODO: make sure image taking functions (specifically darks) aren't taking the same images multiple times in a cube
# TODO: make a function to automatically find the center of the beam and the radius. Use this to crop the image around the beam and only use those pixels

In [39]:
# IT LOOKS LIKE THE IMAGES AREN'T CHANGING IN THE CUBE FOR DARKS!!

In [103]:
# Test making a function to look for saturated pixels
# img_file = r"D:\desktop_drrp_data\test_12_17_2024\darks\-40C_tint_1_FPS_10.fits"
img_file = r"D:\desktop_drrp_data\test_12_17_2024\JHK_waveplate_measurement\middle_position\Cal_1100_Raw\DRRP_JHK_Middle_Position_1100nm_FPS_10_tint_99_QWP1_position_20_QWP2_position_100.fits"
fits.open(img_file)
dark = fits.getdata(img_file)
print(dark)

[[61147.     9.     0. ...  1711.  1859.  2548.]
 [ 1964.  2052.  1634. ...  1565.  6986.  4505.]
 [ 1905.  1570.  1629. ...  8112.  1573.  1928.]
 ...
 [ 1927.  1620.  1622. ...  1828.  1828.  1964.]
 [ 1926.  1477.  1539. ...  1573.  1697.  1697.]
 [ 2117.  1880.  1870. ...  1756.  1811.  2352.]]


In [99]:
img_file = r"D:\desktop_drrp_data\test_12_17_2024\darks\-40C_tint_1_FPS_10.fits"

In [100]:
with fits.open(img_file) as hdul:
        img_data = hdul[0].data
        hdul.close()
        img_median = np.median(img_data, axis=0)
        print(img_median)
        print(img_median.shape)

[[36842.     5.     0. ...   951.  1016.  1109.]
 [ 1004.   991.  1071. ...   992.  1075.  1051.]
 [ 1059.  1038.  1127. ...  1141.  1057.  1168.]
 ...
 [ 1201.  1153.  1229. ...  1112.  1112.  1144.]
 [ 1191.  1065.  1122. ...  1102.  1031.  1031.]
 [ 1197.  1041.  1061. ...  1109.  1087.  1274.]]
(512, 640)


In [102]:
def find_saturated_pixels(img_file, value=16383):
    """
    This function iterates over all the pixels in a fits image and returns the indices of the pixels that match the given value.
    
    Parameters:
    img_file: Path to the fits file.
    value: The value to search for in the img_median. Defaults to 16,383, which is the saturation limit for the CRED-2.
    
    Returns:
    list: A list of indices (y,x) where the value is found.
    """
    saturated_indices = []

    # Works if img_file is a cube of images, not just one frame
    with fits.open(img_file) as hdul:
        img_data = hdul[0].data
        hdul.close()
        img_median = np.median(img_data, axis=0)
        print(img_median.shape)

        for i in range(img_median.shape[0]):
            for j in range(img_median.shape[1]):
                if img_median[i, j] == value:
                    saturated_indices.append((i, j))

    # returns indices in the form (y,x)
    return saturated_indices

In [ ]:
find_saturated_pixels(img_file, 5)

(640,)


IndexError: tuple index out of range

In [111]:
def find_circles(image_file):
    """Find the center and radius of a circular bright spot.
    """
    with fits.open(img_file) as hdul:
        img_data = hdul[0].data
        hdul.close()
        # img_median = np.median(img_data, axis=0)
        # print(img_median.shape)

    # Step 1: Check the shape of the image data
    print(f"Image data shape: {img_data.shape}")
    
    # Step 2: Preprocess the image (apply thresholding or smoothing)
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8, 8))
    enhanced_image = clahe.apply(img_data.astype(np.uint8))
    blurred_image = cv2.GaussianBlur(enhanced_image, (15, 15), 0)
    
    # Step 4: Threshold the image (adjust thresholding for faint circles)
    _, thresholded_image = cv2.threshold(blurred_image, np.max(blurred_image) * 0.3, 255, cv2.THRESH_BINARY)

    # Step 5: Apply Hough Circle Transform with adjusted parameters
    circles = cv2.HoughCircles(thresholded_image.astype(np.uint8),
                               cv2.HOUGH_GRADIENT, dp=1.2, minDist=30,
                               param1=100, param2=20, minRadius=10, maxRadius=100)

    if circles is not None:
        circles = np.round(circles[0, :]).astype("int")
        for circle in circles:
            center = (circle[0], circle[1])
            radius = circle[2]
            print(f"Circle center: {center}, radius: {radius}")
            
            # Optionally, draw the circle
            output_image = img_data.copy()
            cv2.circle(output_image, center, radius, (0, 255, 0), 4)  # Draw the circle
            cv2.rectangle(output_image, (center[0] - 5, center[1] - 5),
                          (center[0] + 5, center[1] + 5), (0, 128, 255), -1)  # Draw the center
            
            # Display the result
            plt.imshow(output_image, cmap='gray')
            plt.show()
    else:
        print("No circles detected.")

In [83]:
with fits.open(img_file) as hdul:
    img_data = hdul[0].data
    hdul.close()
    img_median_file = np.median(img_data, axis=0)

In [115]:
img_file = r"C:\Users\EPL User\Desktop\desktop_drrp_data\test_12_17_2024\JHK_waveplate_measurement\middle_position\Cal_1950_Raw\DRRP_JHK_Middle_Position_1950nm_FPS_10_tint_3_QWP1_position_96_QWP2_position_480.fits"
find_circles(img_file)

Image data shape: (512, 640)
No circles detected.


# Looking at calibration data

In [25]:
# reduced_filename = 'Reduced_DRRP_'
reduced_filename = 'DRRP_'
reduced_folder = "D:\\desktop_drrp_data\\test_12_17_2024\\calibration\\calibration_raw\\Cal_1400_Raw"
lcenter = [200, 194]
rcenter = [200, 391]
maxradius = 65
cutoff = 10000000

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Cal_Il_1400 = extracted_data[0]
Cal_Ir_1400 = extracted_data[1]
Cal_theta1400 = extracted_data[2]

In [27]:
print(Cal_Il_1400)
print(len(Cal_Il_1400))

[ 7754022.   8627872.  24838492.  29274311.  17163783.   2866031.
  2162686.   8603764.  10669063.5 12693247.  25721732.  46651370.
 57914908.  50230602.  31578882.  15669874.   4683261.    603802.
 10508571.  32904224.  47037706.  39099467.  16019697.   4803596.
 15499917.5 28945411.  26038504.   9170305.    763562.   5674189.
 10846550.  11183696.  17351642.  36098698.  55329966.  56867036.
 41252654.5 23216429.   9533785.5  1364930.   3439137.  20215361.
 41905459.  46155560.  28712818.   7921039. ]
46


# Looking at sample data

data_folder = ""D:\desktop_drrp_data\test_12_17_2024\JHK_waveplate_measurement\middle_position\Cal_1400_Raw"

In [29]:
reduced_filename = 'DRRP_'
reduced_folder = "D:\\desktop_drrp_data\\test_12_17_2024\\JHK_waveplate_measurement\\middle_position\\Cal_1400_Raw"
lcenter = [200, 194]
rcenter = [200, 391]
maxradius = 16
cutoff = 90000

extracted_data = extract_intensities(reduced_filename, reduced_folder, lcenter, rcenter, maxradius, cutoff)
Il_1400 = extracted_data[0]
Ir_1400 = extracted_data[1]
theta1400 = extracted_data[2]

In [30]:
data1850 = q_ultimate_polarimetry(Cal_theta1400, Cal_Il_1400, Cal_Ir_1400, theta1400, Il_1400, Ir_1400)
print(data1850[0], "This is the Mueller matrix of the sample waveplate")
print(data1850[1], "This is the retardance of the sample")
print(data1850[2], "This is the Mueller matrix for calibration (close to identity)")
print(data1850[3], "This is the RMS error in the calibration matrix")
print(data1850[4], "This is the retardance error in the sample waveplate")

[-0.09450612 -0.25587673 -0.17918556 -0.02936682 -0.04962122] Fit parameters for a1, w1, w2, r1, and r2. 1 for generator, 2 for analyzer
[[ 0.76765311  0.01587117 -0.09661647 -0.01138913]
 [ 0.16697467  0.02960208 -0.86084522 -0.03613853]
 [ 0.11590009 -1.          0.07945278 -0.02407883]
 [ 0.00537916  0.01045846  0.00350263 -0.80047537]] This is the Mueller matrix of the sample waveplate
0.45372425143462447 This is the retardance of the sample
[[ 0.89676117 -0.02458228  0.01767613 -0.00429988]
 [-0.06243619  0.92074268  0.07405617 -0.00324003]
 [ 0.01441358 -0.03792484  1.         -0.00182594]
 [-0.00239727 -0.00221436 -0.01226491  0.89208936]] This is the Mueller matrix for calibration (close to identity)
0.050450133948103014 This is the RMS error in the calibration matrix
0.1759809311231941 This is the retardance error in the sample waveplate


# Analysis

Dodge for MgF2, Ghosh for SiO2 (quartz)

In [ ]:
n_e = np.loadtxt("/home/shared/exoserver/Lab_Data/Mueller_Matrix_Polarimeter/L_Plate_Characterization/SuperK_Select_Data/Dodge-e.csv", delimiter=",",skiprows=1)
n_o = np.loadtxt("/home/shared/exoserver/Lab_Data/Mueller_Matrix_Polarimeter/L_Plate_Characterization/SuperK_Select_Data/Dodge-o.csv", delimiter=",",skiprows=1)